In [4]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from implementations import *
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y_train, X_train, ids = load_csv_data(DATA_TRAIN_PATH)

In [6]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [39]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0.3,
                                                                 encodable_min_th=0.3, encodable_max_th=0.7, switch_encoding=True)

In [40]:
tX_train

array([[ 1.        ,  0.06833197,  0.40768027, ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  0.55250482,  0.54013641, ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  3.19515553,  1.09655998, ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [ 1.        ,  0.31931645, -0.13086367, ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        , -0.84532397, -0.30297338, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.66533608, -0.25352276, ...,  0.        ,
         0.        ,  0.        ]])

In [41]:
tX_train.shape, tX_test.shape

((250000, 24), (568238, 24))

In [44]:
param_grid = {
    'lambda_': np.logspace(-4, 0, 5),
    'degree': list(range(1, 4)),
    'max_iters': 100,
    'cont_features': [cont_features]
}
metrics, params = logistic_regression_cv(ty_train, tX_train, param_grid=param_grid)

In [46]:
metrics

{'loss': 0.48832368060485054,
 'accuracy': 76.5648,
 'f1_score': 0.6261136027521852}

In [47]:
from implementations import reg_logistic_regression, build_poly
tX_train_poly = build_poly(tX_train, degree=params['degree'], cont_features=cont_features)
weights, loss = reg_logistic_regression(ty_train, tX_train_poly, max_iters=1000, lambda_=params['lambda_'])

In [49]:
from helpers import predict_logistic, compute_accuracy, compute_f1
ty_train_pred = predict_logistic(weights, tX_train_poly)
train_accuracy = compute_accuracy(ty_train, ty_train_pred)
train_f1 = compute_f1(ty_train, ty_train_pred)
train_accuracy, train_f1

(78.54560000000001, 0.6573436401967674)

In [50]:
tX_test.shape

(568238, 24)

In [51]:
tX_test_poly = build_poly(tX_test, params['degree'], cont_features)

In [52]:
tX_test_poly.shape

(568238, 44)

In [54]:
from datetime import datetime
method = 'reg_logistic_regression'
time = datetime.now().strftime('%Y%m%dH%H%M%S')
OUTPUT_PATH = f'../submissions/submission_{method}_{time}'
y_pred = predict_logistic(weights, tX_test_poly)
y_pred = replace_values(y_pred, from_val=0, to_val=-1)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)